In [ ]:
import pandas as pd
import plotly as px

In [ ]:
def read_fixture(fixture: str, rule='T'):
    #Legge CSV
    df = pd.read_csv(f'./data/feed_{fixture}.MYD.csv', delimiter=' ', header=None, names=['date', fixture])
    df['date'] = pd.to_datetime(df['date'], unit='s')
    # Rinomina la colonna fixture in 'consumption'
    df.rename(columns={fixture: 'consumption'}, inplace=True)
    df.set_index('date', inplace=True)
    # Resample il DataFrame usando la frequenza specificata (minutale 'T')
    df_minutely = df.resample(rule).sum()
    
    # Ora raggruppiamo i dati minutali in giornate
    df_daily = df_minutely.resample('D').sum()
    
    # Creare un range di date da 1/1/2019 a 31/1/2019
    date_rng = pd.date_range(start='13/02/2019', end='28/02/2019', freq='D')
    
    # Creare un DataFrame vuoto con il range di date specificato
    df_final = pd.DataFrame(date_rng, columns=['date'])
    df_final.set_index('date', inplace=True)
    
    # Unire il DataFrame giornaliero con il DataFrame vuoto per assicurarsi che tutte le date siano presenti
    df_final = df_final.join(df_daily, how='left').fillna(0)
    
    return df_final

In [ ]:
def range_plot(df):
    fig = px.line(df, y='consumption', labels={'consumption': 'Volume [L]'})
    fig.update_xaxes()
    fig.update_yaxes(fixedrange=False)
    fig.update_layout(font=dict(size=22))#grandezza font
    fig.show()

In [ ]:
df_washbasin = read_fixture('Washbasin')
display(df_washbasin)
df_washbasin.plot()